<a href="https://colab.research.google.com/github/sathsaranir1996/Rice-Seed-Identification-System/blob/main/image_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import required libraries(OpenCV, NumPy, Pandas, Matplotlib, TenserFlow, os, csv, globe)
import os
import cv2
import csv
import glob
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from PIL import Image

folder_path = 'drive/My Drive/images/dikwee'

# Get all files in the folder
files = glob.glob(os.path.join(folder_path, '*'))

# Filter out non-image files
image_files = [file for file in files if file.endswith(('.png', '.jpg', '.jpeg', '.gif'))]

# Load the images
images = []
for image_file in image_files:
    with open(image_file, 'rb') as f:
        images.append(f.read())

        # Reading the image
        image = cv2.imread(image_file)

        print(image_file)
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        plt.title("Original Image")
        plt.show()

        # Crop the center of the image using TenserFlow (Fraction = 90%)
        cropped_img = tf.image.central_crop(image, central_fraction=0.9).numpy()

        plt.imshow(cv2.cvtColor(cropped_img, cv2.COLOR_BGR2RGB))
        plt.title("Cropped Image")
        plt.show()

        # Resizing the image
        resized_img = cv2.resize(cropped_img, (600, 400))

        plt.imshow(cv2.cvtColor(resized_img, cv2.COLOR_BGR2RGB))
        plt.title("Resized Image")
        plt.show()

        # Converting the image to grayscale
        gray_img = cv2.cvtColor(resized_img, cv2.COLOR_BGR2GRAY)

        plt.imshow(gray_img, cmap="gray")
        plt.title("Gray Image")
        plt.show()

        # Applying thresholding (Otsu's thresholding)
        ret, thresh_img = cv2.threshold(gray_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        plt.imshow(thresh_img)
        plt.title("Image after Thresholding")
        plt.show()

        # Morphological operations
        kernel = np.ones((3,3),np.uint8)
        morph_img = cv2.erode(thresh_img,kernel,iterations = 1)
        morph_img = cv2.dilate(morph_img,kernel,iterations = 1)

        # Feature extraction
        canny_edges = cv2.Canny(morph_img, threshold1=30, threshold2=100)
        contours, hierarchy = cv2.findContours(canny_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        plt.imshow(canny_edges, cmap="gray")
        plt.title("Image after Canny Edge Detection")
        plt.show()

        # Get the count of contours
        count = len(contours)
        print('Count of contours:', count)
        print('----------------------------')

        # Initialize variables
        total_area = 0
        total_perimeter = 0
        lengths = []
        widths = []
        areas = []
        perimeters = []
        colors = []
        shapes = []
        seed_type = 'Dikwee'

        # Loop through the contours and compute their area and perimeter
        for contour in contours:

            # Get the minimum area rectangle that bounds the contour
            rect = cv2.minAreaRect(contour)

            # Extract the length and width of the rectangle
            length, width = rect[1]

            # Print the length and width
            print('Length:', length)
            print('Width:', width)

            area = cv2.contourArea(contour)
            perimeter = cv2.arcLength(contour, True)

            # Appending the calculated length, width, area and perimiter to separate lists
            lengths.append(length)
            widths.append(width)
            areas.append(area)
            perimeters.append(perimeter)

            print('Area:', area)
            print('Perimeter:', perimeter)

            total_area += area
            total_perimeter += perimeter

            # Creating multiple masks using cv2 boundingrect
            x,y,w,h = cv2.boundingRect(contour)
            masked_img = cv2.rectangle(resized_img,(x,y),(x+w,y+h),(0,255,0),2)

            # Compute the mean color value of the contour
            mask = np.zeros_like(gray_img)
            cv2.drawContours(mask, [contour], 0, 255, -1)
            mean_color = cv2.mean(gray_img, mask=mask)

            # Appending the calculated mean color to a list named colors.
            colors.append(mean_color)

            print('Contour color:', mean_color)

            # Calculate the contour shape
            contour_shape = contour.shape

            # Appending the calculated shape to a list named shapes.
            shapes.append(contour_shape)

            print('Shape:', contour_shape, '\n')


        # Convert the area and perimeter lists to a pandas DataFrame
        df = pd.DataFrame({'Length': lengths, 'Width': widths, 'Area': areas, 'Perimeter': perimeters, 'Color': colors, 'Shape': shapes, 'Class' : seed_type})

        img_file_name = os.path.basename(image_file).split('.')[0]

        # Save the DataFrame to a CSV file with a unique file name
        csv_file_name = 'seed_data_' + str(img_file_name) + '.csv'
        df.to_csv(csv_file_name, index=False)

        plt.imshow(cv2.cvtColor(masked_img, cv2.COLOR_BGR2RGB))
        plt.title("Image after creating multiple masks")
        plt.show()

        # Compute the average area and perimeter
        avg_area = total_area / len(contours)
        avg_perimeter = total_perimeter / len(contours)

        print('----------------------------------')
        print('Average Area:', avg_area)
        print('Average Perimeter:', avg_perimeter)
        print('----------------------------------')


print(f"Loaded {len(images)} images")
